In [1]:
# By Ismael Valenzuela (@aboutsecurity / @thinkredactblue)
# BlackBerry Threat Research & Intelligence team 

import pandas as pd
import plotly.graph_objects as go

In [2]:

# This function takes a file with the mappings source country, threat actor, and targeted industry and generates a sankey with these relations

def threatmodel(file_name: str, country: str):
    
    data = pd.read_csv(file_name)
    unique_source_target = list(pd.unique(data[['source','target']].values.ravel('k')))

    mapping_dict = {k: v for v, k in enumerate(unique_source_target)}

    data['source'] = data['source'].map(mapping_dict)
    data['target'] = data['target'].map(mapping_dict)

    links_dict = data.to_dict(orient='list')   

    fig = go.Figure(data=[go.Sankey(
    node = dict(
        pad = 15,
        thickness=20,
        line=dict(color='black', width=1),
        label = unique_source_target,
        color='green'
    ),
    link = dict(
        source = links_dict['source'],
        target = links_dict['target'],
        value = links_dict['value']
    ),
    textfont=dict(color="black", size=16)
    )])
    fig.update_layout(title='Threat Actors Targeting ' + country + ' per Industry', 
                        font_size=16,
                        autosize=True,
                        width=1000,
                        height=800)

    fig.add_annotation(dict(font=dict(color="black",size=12), x=0, y=1.06, showarrow=False, text='<b>Sponsor</b>'))
    fig.add_annotation(dict(font=dict(color="black",size=12), x=0.5, y=1.06, showarrow=False, text='<b>Actor</b>'))
    fig.add_annotation(dict(font=dict(color="black",size=12), x=1, y=1.07, showarrow=False, text='<b>Industry</b>'))

    fig.show() 
    

In [3]:
threatmodel('chile-threat_actors-v2.csv', 'Chile')
threatmodel('ecuador-threat_actors-v2.csv', 'Ecuador')
threatmodel('colombia-threat_actors-v2.csv', 'Colombia')
threatmodel('argentina-threat_actors-v2.csv', 'Argentina')
threatmodel('mexico-threat_actors-v2.csv', 'Mexico')
threatmodel('brasil-threat_actors-v2.csv', 'Brasil')
threatmodel('spain-threat_actors-v2.csv', 'Spain')


In [6]:
# This code takes a CSV with a list of TTPs, aggregates them and generates a treemap visualization

import plotly.express as px
import numpy

data = pd.read_csv('attack-mappings-ttps.csv')
newdata = data.groupby(['Techniques']).size().reset_index(name='counts').sort_values(['counts'], ascending=False)

def set_color(row):
    if row["counts"] == 4:
        return "red"
    elif row["counts"] == 3:
        return "orange"
    elif row["counts"] == 2:
        return "yellow"
    else:
        return "lightblue"

newdata = newdata.assign(color=newdata.apply(set_color, axis=1))

# Parses text and prepare it so it displays well in the chart

newdata['Techniques']= newdata['Techniques'].replace('-','', regex=True)
newdata['Techniques']= newdata['Techniques'].replace('–','', regex=True)
newdata['Techniques']= newdata['Techniques'].replace(' ','<br>', regex=True) 
 
fig = px.treemap(newdata, path=['Techniques'],
                 values='counts',
                 color='color',
                 color_discrete_map={ 'red':'red','orange':'orange','yellow':'yellow','lightblue':'lightblue'}
                 )

fig.update_traces(root_color="grey", textfont_size=14)

fig.update_layout(
    height=800,
    title_text='Most common TTPs - All Countries',
)
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))


In [16]:
# Extract top 11 TTPs and map to D3FEND

newdata['TTP'] = newdata['Techniques'].str.split('<br>').str[0]

ttp_list = newdata['TTP']
ttp_list = ttp_list.iloc[0:11]

ttp_list.astype(str).values.flatten().tolist()
print (ttp_list.values.tolist())

# MITRE D3FEND extractor mapping

# https://d3fend.mitre.org/tools/attack-extractor/?q=%5B%22T1204.002%22%2C%22T1071.001%22%2C%22T1059.001%22%2C%22T1486%22%2C%22T1047%22%2C%22T1133%22%2C%22T1490%22%2C%22T1105%22%2C%22T1547.001%22%2C%22T1003%22%2C%22T1568%22%5D

['T1204.002', 'T1059.001', 'T1486', 'T1133', 'T1047', 'T1071.001', 'T1490', 'T1505.003', 'T1105', 'T1547.001', 'T1003']
